In [1]:
from openai import OpenAI

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key="sk-0rLvuRkMiD4Mw25QYygh6rUlZVjpQWNGNF4yez7z3PZ7yCOm",
    # api_key="sk-tMbkq3K1iO5vf0FRMlrmzslGXJZwE0us3mve4QXuvpnZcumG",
    base_url="https://api.chatanywhere.cn/v1"
    # base_url="https://api.chatanywhere.cn/v1"
)

In [2]:
def gpt_35_api(messages: list):

    completion = client.chat.completions.create(model="gpt-3.5-turbo-0125", messages=messages, temperature=0)
    return completion.choices[0].message.content

In [3]:
import json
from tqdm import tqdm

In [4]:
# from langchain import hub
# 
# prompt = hub.pull("tyfann/llm4commit-rag:b843ef0b")

from langchain import hub
prompt = hub.pull("tyfann/llm4commit-rag:b843ef0b")

In [5]:
# for model in ['mxbai', 'miniLM']:
for model in ['e5']:
    with open(f'../data/angular_filtered/subsets/generation/rag_prompt/dev_test_rag_{model}_prompt.json', 'r', encoding='UTF-8') as f:
        org_data = json.load(f)
    gpt_msg = []
    for index, data in tqdm(enumerate(org_data), total=len(org_data), desc="Processing documents"):
        # merged_diff = '\n'.join(diff['diff'] for diff in data['diff'])
        messages = prompt.invoke(
            {"context": data['sim_diff'], "msg": data['sim_msg'], "diff": data['diff']}
        ).to_messages()
        example_prompt = [{'role': 'user','content': messages[0].content},]
        try:
            gpt_msg.append(gpt_35_api(example_prompt))
        except:
            print(index)
            gpt_msg.append("")
    
    for item, msg in zip(org_data, gpt_msg):
        item['chatgpt_rag'] = msg
    
    output_file = f'../data/angular_filtered/subsets/generation/dev_test_gpt35_rag_{model}.json'
    with open(output_file, 'w', encoding='UTF-8') as f:
        json.dump(org_data, f, ensure_ascii=False, indent=4)
        
    output_file = f'../data/angular_filtered/subsets/generation/dev_test_gpt35_rag_{model}.txt'
    with open(output_file, 'w', encoding='UTF-8') as f:
        for item in org_data:
            f.write(item['chatgpt_rag'].replace('\n', '\\n').replace('\r', '\\r') + '\n')

Processing documents: 100%|██████████| 603/603 [07:38<00:00,  1.32it/s]


In [13]:
output_file = '../data/angular_filtered/test_result/test_data_js_gpt35_hybrid_golden.txt'
with open(output_file, 'w', encoding='UTF-8') as f:
    for item in org_data:
        f.write(item['chatgpt_hybrid'] + '\n')

In [6]:
gpt_msg = []
for index, data in tqdm(enumerate(org_data), total=len(org_data), desc="Processing documents"):
    messages = prompt.invoke(
        {"context": data['sim_diff'], "msg": data['sim_msg'], "diff": data['diff']}
    ).to_messages()
    example_prompt = [{'role': 'user','content': messages[0].content},]
    try:
        gpt_msg.append(gpt_35_api(example_prompt))
    except:
        print(index)
        gpt_msg.append("")

for item, msg in zip(org_data, gpt_msg):
    item['chatgpt_rag'] = msg

output_file = '../data/angular_filtered/test_result/test_data_js_gpt35_rag_golden.json'
with open(output_file, 'w', encoding='UTF-8') as f:
    json.dump(org_data, f, ensure_ascii=False, indent=4)

output_file = '../data/angular_filtered/test_result/test_data_js_gpt35_rag_golden.txt'
with open(output_file, 'w', encoding='UTF-8') as f:
    for item in org_data:
        f.write(item['chatgpt_rag'] + '\n')

Processing documents: 100%|██████████| 505/505 [06:32<00:00,  1.29it/s]


In [19]:
output_file = '../data/angular_filtered/test_result/test_data_js_gpt35_rag_mxbai.txt'
with open(output_file, 'w', encoding='UTF-8') as f:
    for item in org_data:
        f.write(item['chatgpt_rag'] + '\n')

In [20]:
with open('../data/angular_filtered/test_result/test_data_js_gpt35_rag_miniLM.json', 'r', encoding='UTF-8') as f:
    org_data = json.load(f)
# save msg to txt
output_file = '../data/angular_filtered/test_result/test_data_js_gpt35_rag_miniLM.txt'
with open(output_file, 'w', encoding='UTF-8') as f:
    for item in org_data:
        f.write(item['chatgpt_rag'] + '\n')

In [5]:
# TODO
lang = 'js'
with open(f'../data/final_preprocessed_data/{lang}_baseline/{lang}_baseline_weaviate_rag_300_prompt.json', 'r', encoding='UTF-8') as f:
    org_data = json.load(f)

org_data = org_data[:100]
gpt_msg = []
for index, data in tqdm(enumerate(org_data), total=len(org_data), desc="Processing documents"):
    messages = prompt.invoke(
        {"context": data['sim_diff'], "msg": data['sim_msg'], "diff": data['org_diff']}
    ).to_messages()
    example_prompt = [{'role': 'user','content': messages[0].content},]
    try:
        msg = gpt_35_api(example_prompt)
        while msg is None:
            msg = gpt_35_api(example_prompt)
        gpt_msg.append(msg)
    except:
        print(index)
        gpt_msg.append("")

with open(f'../data/final_preprocessed_data/{lang}_baseline_test_data_300.json', 'r', encoding='UTF-8') as f:
    chronicle_data = json.load(f)[:100]

for item, msg in zip(chronicle_data, gpt_msg): 
    item['chatgpt_rag'] = msg

output_file = f'../data/final_preprocessed_data/{lang}_baseline/{lang}_baseline_weaviate_rag_300_dev.json'
with open(output_file, 'w', encoding='UTF-8') as f:
    json.dump(chronicle_data, f, ensure_ascii=False, indent=4)

In [5]:
# TODO
prompt = hub.pull("tyfann/llm4commit-rag:b843ef0b")
lang = 'js'
with open(f'../data/final_preprocessed_data/{lang}_baseline/{lang}_baseline_test_data_300_e5.json', 'r', encoding='UTF-8') as f:
    org_data = json.load(f)[:100]

# org_data = org_data[:100]
gpt_msg = []
for index, data in tqdm(enumerate(org_data), total=len(org_data), desc="Processing documents"):
    messages = prompt.invoke(
        {"context": data['sim_diff'], "msg": data['sim_msg'], "diff": data['org_diff']}
    ).to_messages()
    example_prompt = [{'role': 'user','content': messages[0].content},]
    try:
        msg = gpt_35_api(example_prompt)
        while msg is None:
            msg = gpt_35_api(example_prompt)
        gpt_msg.append(msg)
    except:
        print(index)
        gpt_msg.append("")

with open(f'../data/final_preprocessed_data/{lang}_baseline_test_data_300.json', 'r', encoding='UTF-8') as f:
    chronicle_data = json.load(f)[:100]

for item, msg in zip(chronicle_data, gpt_msg): 
    item['chatgpt_rag'] = msg

output_file = f'../data/final_preprocessed_data/{lang}_baseline/{lang}_baseline_test_data_300_e5_chatgpt.json'
with open(output_file, 'w', encoding='UTF-8') as f:
    json.dump(chronicle_data, f, ensure_ascii=False, indent=4)

In [8]:
# TODO
prompt = hub.pull("tyfann/llm4commit-rag:b843ef0b")
lang = 'js'
with open(f'../data/final_preprocessed_data/{lang}_baseline/{lang}_baseline_test_data_300_e5_finetune_10w.json', 'r', encoding='UTF-8') as f:
    org_data = json.load(f)[:100]

# org_data = org_data[:100]
gpt_msg = []
for index, data in tqdm(enumerate(org_data), total=len(org_data), desc="Processing documents"):
    messages = prompt.invoke(
        {"context": data['sim_diff'], "msg": data['sim_msg'], "diff": data['org_diff']}
    ).to_messages()
    example_prompt = [{'role': 'user','content': messages[0].content},]
    try:
        msg = gpt_35_api(example_prompt)
        while msg is None:
            msg = gpt_35_api(example_prompt)
        gpt_msg.append(msg)
    except:
        print(index)
        gpt_msg.append("")

with open(f'../data/final_preprocessed_data/{lang}_baseline_test_data_300.json', 'r', encoding='UTF-8') as f:
    chronicle_data = json.load(f)[:100]

for item, msg in zip(chronicle_data, gpt_msg): 
    item['chatgpt_rag'] = msg

output_file = f'../data/final_preprocessed_data/{lang}_baseline/{lang}_baseline_test_data_300_e5_finetune_10w_chatgpt.json'
with open(output_file, 'w', encoding='UTF-8') as f:
    json.dump(chronicle_data, f, ensure_ascii=False, indent=4)

In [8]:
len(gpt_msg)

In [35]:
lang = 'js'
with open(f'../data/final_preprocessed_data/{lang}_baseline/{lang}_baseline_rag_300_prompt.json', 'r', encoding='UTF-8') as f:
    org_data = json.load(f)

for data in org_data:
    data['sim_diff'] = data['org_diff']
    data['sim_msg'] = data['org_msg']

org_data = org_data[:100]
gpt_msg = []
for index, data in tqdm(enumerate(org_data), total=len(org_data), desc="Processing documents"):
    messages = prompt.invoke(
        {"context": data['sim_diff'], "msg": data['sim_msg'], "diff": data['org_diff']}
    ).to_messages()
    example_prompt = [{'role': 'user','content': messages[0].content},]
    try:
        msg = gpt_35_api(example_prompt)
        while msg is None:
            msg = gpt_35_api(example_prompt)
        gpt_msg.append(msg)
    except:
        print(index)
        gpt_msg.append("")

with open(f'../data/final_preprocessed_data/{lang}_baseline_test_data_300.json', 'r', encoding='UTF-8') as f:
    chronicle_data = json.load(f)[:100]

for item, msg in zip(chronicle_data, gpt_msg): 
    item['chatgpt_rag'] = msg

output_file = f'../data/final_preprocessed_data/{lang}_baseline/{lang}_baseline_rag_300_dump_2.json'
with open(output_file, 'w', encoding='UTF-8') as f:
    json.dump(chronicle_data, f, ensure_ascii=False, indent=4)

In [37]:
# output_file = f'../data/final_preprocessed_data/{lang}_baseline/{lang}_baseline_zeroshot_300.json'
# with open(output_file, 'r', encoding='UTF-8') as f:
#     org_data = json.load(f)
# 
# org_data = org_data[:100]
# with open(f'../data/final_preprocessed_data/{lang}_baseline/{lang}_baseline_zeroshot_300_dump.json', 'w', encoding='UTF-8') as f:
#     json.dump(org_data, f, ensure_ascii=False, indent=4)
# 
# 
# output_file = f'../data/final_preprocessed_data/{lang}_baseline/{lang}_baseline_race_300.json'
# with open(output_file, 'r', encoding='UTF-8') as f:
#     org_data = json.load(f)
# 
# org_data = org_data[:100]
# with open(f'../data/final_preprocessed_data/{lang}_baseline/{lang}_baseline_race_300_dump.json', 'w', encoding='UTF-8') as f:
#     json.dump(org_data, f, ensure_ascii=False, indent=4)

output_file = f'../data/final_preprocessed_data/{lang}_baseline/{lang}_baseline_rag_300.json'
with open(output_file, 'r', encoding='UTF-8') as f:
    org_data = json.load(f)

org_data = org_data[:100]
with open(f'../data/final_preprocessed_data/{lang}_baseline/{lang}_baseline_rag_300_v3.json', 'w', encoding='UTF-8') as f:
    json.dump(org_data, f, ensure_ascii=False, indent=4)

In [24]:
data = org_data[631]
messages = prompt.invoke(
    {"context": data['sim_diff'], "msg": data['sim_msg'], "diff": data['org_diff']}
).to_messages()
example_prompt = [{'role': 'user','content': messages[0].content},]
print(gpt_35_api(example_prompt))

In [22]:
for index, data in enumerate(org_data):
    if data['org_msg']=="Tooltip: invoke 'doDestory' manually when disabled":
        print(index)

In [26]:
output_file = f'../data/final_preprocessed_data/{lang}_baseline/{lang}_baseline_rag_300.json'
with open(output_file, 'w', encoding='UTF-8') as f:
    json.dump(chronicle_data, f, ensure_ascii=False, indent=4)

In [6]:
lang = 'js'
with open(f'../data/final_preprocessed_data/{lang}_baseline/{lang}_baseline_nochunk_rag_prompt.json', 'r', encoding='UTF-8') as f:
    org_data = json.load(f)

gpt_msg = []
for index, data in tqdm(enumerate(org_data), total=len(org_data), desc="Processing documents"):
    messages = prompt.invoke(
        {"context": data['sim_diff'], "msg": data['sim_msg'], "diff": data['org_diff']}
    ).to_messages()
    example_prompt = [{'role': 'user','content': messages[0].content},]
    try:
        gpt_msg.append(gpt_35_api(example_prompt))
    except:
        print(index)
        gpt_msg.append("")

with open(f'../data/final_preprocessed_data/{lang}_baseline_test_data.json', 'r', encoding='UTF-8') as f:
    chronicle_data = json.load(f)[:1000]

for item, msg in zip(chronicle_data, gpt_msg):
    item['chatgpt_rag'] = msg

output_file = f'../data/final_preprocessed_data/{lang}_baseline/{lang}_baseline_nochunk_rag.json'
with open(output_file, 'w', encoding='UTF-8') as f:
    json.dump(chronicle_data, f, ensure_ascii=False, indent=4)

# 4SIM

In [7]:
prompt = hub.pull("tyfann/llm4commit-rag-4sim")
for lang in ['js']:
    with open(f'../data/final_preprocessed_data/{lang}_baseline/{lang}_baseline_rag_top4_prompt.json', 'r', encoding='UTF-8') as f:
        org_data = json.load(f)
    
    gpt_msg = []
    for index, data in tqdm(enumerate(org_data), total=len(org_data), desc="Processing documents"):
        try:
            messages = prompt.invoke(
                {"diff1": data['sim_diff_0'],  "msg1": data['sim_msg_0'],
                 "diff2": data['sim_diff_1'],  "msg2": data['sim_msg_1'],
                 "diff3": data['sim_diff_2'],  "msg3": data['sim_msg_2'],
                 "diff4": data['sim_diff_3'],  "msg4": data['sim_msg_3'],
                 "question": data['org_diff']}
            ).to_messages()
            example_prompt = [{'role': 'user','content': messages[0].content},]
            gpt_msg.append(gpt_35_api(example_prompt))
        except:
            print(index)
            gpt_msg.append('')
    
    with open(f'../data/final_preprocessed_data/{lang}_baseline_test_data.json', 'r', encoding='UTF-8') as f:
        chronicle_data = json.load(f)[:1000]
    
    for item, msg in zip(chronicle_data, gpt_msg):
        item['chatgpt_rag'] = msg
    
    output_file = f'../data/final_preprocessed_data/{lang}_baseline/{lang}_baseline_rag_top4.json'
    with open(output_file, 'w', encoding='UTF-8') as f:
        json.dump(chronicle_data, f, ensure_ascii=False, indent=4)

# SIM_MSG

In [8]:
prompt = hub.pull("tyfann/llm4commit-rag-msg")
for lang in [ 'js']:
    with open(f'../data/final_preprocessed_data/{lang}_baseline/{lang}_baseline_rag_sim_msg_prompt.json', 'r', encoding='UTF-8') as f:
        org_data = json.load(f)
    
    gpt_msg = []
    for index, data in tqdm(enumerate(org_data), total=len(org_data), desc="Processing documents"):
        try:
            messages = prompt.invoke(
                {"diff1": data['sim_diff'],  "msg1": data['sim_msg'],
                 "question": data['org_diff'], 'draft': data['org_msg']}
            ).to_messages()
            example_prompt = [{'role': 'user','content': messages[0].content},]
            gpt_msg.append(gpt_35_api(example_prompt))
        except:
            print(index)
            gpt_msg.append('')
    
    
    with open(f'../data/final_preprocessed_data/{lang}_baseline_test_data.json', 'r', encoding='UTF-8') as f:
        chronicle_data = json.load(f)[:1000]
    
    for item, msg in zip(chronicle_data, gpt_msg):
        item['chatgpt_rag'] = msg
    
    output_file = f'../data/final_preprocessed_data/{lang}_baseline/{lang}_baseline_rag_sim_msg.json'
    with open(output_file, 'w', encoding='UTF-8') as f:
        json.dump(chronicle_data, f, ensure_ascii=False, indent=4)